# Error Handling in SQLite Operations

In this notebook, you'll learn how to handle errors and exceptions that can occur when working with SQLite databases in Python. Proper error handling is crucial for robust database applications.

In [ ]:
import sqlite3
import os

# Connect to database
conn = sqlite3.connect('error_handling_demo.db')
cursor = conn.cursor()

# Create a test table
cursor.execute('''
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        username TEXT UNIQUE NOT NULL,
        email TEXT UNIQUE,
        age INTEGER CHECK(age >= 0 AND age <= 150)
    )
''')

print('Database and table created')

## SQLite Exception Hierarchy

SQLite errors in Python are represented by the `sqlite3.Error` exception class and its subclasses:

- `sqlite3.Error`: Base class for all SQLite errors
  - `sqlite3.DatabaseError`: Errors related to the database
    - `sqlite3.IntegrityError`: Constraint violations (UNIQUE, NOT NULL, CHECK, etc.)
    - `sqlite3.OperationalError`: Database operation errors (disk full, locked database, etc.)
  - `sqlite3.ProgrammingError`: SQL syntax errors, wrong number of parameters, etc.
  - `sqlite3.NotSupportedError`: Unsupported operations

Always catch `sqlite3.Error` to handle all SQLite-related exceptions.

## Basic Error Handling

Use try-except blocks to catch and handle database errors:

In [ ]:
# Example of basic error handling
try:
    cursor.execute('INSERT INTO users (username, email, age) VALUES (?, ?, ?)', 
                   ('alice', 'alice@example.com', 25))
    conn.commit()
    print('User inserted successfully')
except sqlite3.Error as e:
    print(f'Database error: {e}')
    conn.rollback()
except Exception as e:
    print(f'Unexpected error: {e}')
    conn.rollback()

## Handling Constraint Violations

Different types of constraint violations raise different exceptions:

In [ ]:
# Test different constraint violations
test_cases = [
    ('UNIQUE username', ('alice', 'different@example.com', 30)),  # Duplicate username
    ('UNIQUE email', ('bob', 'alice@example.com', 30)),         # Duplicate email
    ('NOT NULL', (None, 'charlie@example.com', 30)),            # NULL username
    ('CHECK constraint', ('diana', 'diana@example.com', -5)),     # Invalid age
    ('CHECK constraint', ('eve', 'eve@example.com', 200))        # Age too high
]

for error_type, data in test_cases:
    try:
        cursor.execute('INSERT INTO users (username, email, age) VALUES (?, ?, ?)', data)
        print(f'{error_type}: Insert succeeded (unexpected)')
    except sqlite3.IntegrityError as e:
        print(f'{error_type}: IntegrityError - {e}')
    except sqlite3.Error as e:
        print(f'{error_type}: Other SQLite error - {e}')
    
    # Rollback any partial changes
    conn.rollback()

## Handling Operational Errors

Operational errors occur due to database conditions:

In [ ]:
# Test operational errors
try:
    # Try to create a table that already exists
    cursor.execute('CREATE TABLE users (id INTEGER)')
except sqlite3.OperationalError as e:
    print(f'OperationalError: {e}')

# Test with invalid SQL
try:
    cursor.execute('INVALID SQL SYNTAX')
except sqlite3.OperationalError as e:
    print(f'OperationalError (invalid SQL): {e}')
except sqlite3.ProgrammingError as e:
    print(f'ProgrammingError (invalid SQL): {e}')

## Connection Errors

Handle errors that occur when establishing connections:

In [ ]:
# Test connection to non-existent database file (should work - SQLite creates it)
try:
    test_conn = sqlite3.connect('test.db')
    print('Connection to new database successful')
    test_conn.close()
    os.remove('test.db')  # Clean up
except sqlite3.Error as e:
    print(f'Connection error: {e}')

# Test connection with invalid permissions (if possible)
# Note: On Windows, this might not trigger an error
try:
    # Try to connect to a directory (should fail)
    bad_conn = sqlite3.connect('C:\\')
    print('Unexpectedly connected to directory')
    bad_conn.close()
except sqlite3.OperationalError as e:
    print(f'OperationalError (invalid path): {e}')
except sqlite3.Error as e:
    print(f'Other connection error: {e}')

## Transaction Error Handling

Proper error handling in transactions is critical:

In [ ]:
def safe_transaction_operation():
    """Demonstrate proper transaction error handling"""
    try:
        # Start transaction
        conn.execute('BEGIN TRANSACTION')
        
        # Perform operations that might fail
        cursor.execute('INSERT INTO users (username, email, age) VALUES (?, ?, ?)', 
                       ('transaction_user', 'tx@example.com', 25))
        
        # This will fail due to duplicate username
        cursor.execute('INSERT INTO users (username, email, age) VALUES (?, ?, ?)', 
                       ('alice', 'different@example.com', 30))
        
        # If we get here, commit
        conn.commit()
        print('Transaction committed successfully')
        return True
        
    except sqlite3.IntegrityError as e:
        # Rollback on constraint violation
        conn.rollback()
        print(f'Integrity error, transaction rolled back: {e}')
        return False
        
    except sqlite3.OperationalError as e:
        # Rollback on operational error
        conn.rollback()
        print(f'Operational error, transaction rolled back: {e}')
        return False
        
    except sqlite3.Error as e:
        # Rollback on any other SQLite error
        conn.rollback()
        print(f'Database error, transaction rolled back: {e}')
        return False
        
    except Exception as e:
        # Rollback on unexpected errors
        conn.rollback()
        print(f'Unexpected error, transaction rolled back: {e}')
        return False

print('Safe transaction function defined')

# Test the function
result = safe_transaction_operation()
print(f'Operation result: {result}')

# Verify no partial data was committed
cursor.execute('SELECT COUNT(*) FROM users WHERE username = "transaction_user"')
count = cursor.fetchone()[0]
print(f'Users with username "transaction_user": {count} (should be 0 due to rollback)')

## Using Context Managers

Context managers provide automatic error handling and cleanup:

In [ ]:
def database_operation_with_context():
    """Use context manager for automatic transaction handling"""
    try:
        with conn:
            # This automatically starts a transaction
            cursor.execute('INSERT INTO users (username, email, age) VALUES (?, ?, ?)', 
                           ('context_user', 'context@example.com', 28))
            
            # Simulate an error
            # cursor.execute('INSERT INTO nonexistent_table VALUES (1)')
            
            print('Context manager: Operation completed successfully')
            return True
            
    except sqlite3.Error as e:
        print(f'Context manager: Database error (automatically rolled back): {e}')
        return False
    
print('Context manager function defined')

# Test successful operation
result = database_operation_with_context()
print(f'Context manager result: {result}')

# Check if user was inserted
cursor.execute('SELECT COUNT(*) FROM users WHERE username = "context_user"')
count = cursor.fetchone()[0]
print(f'Users with username "context_user": {count}')

## Custom Error Handling Classes

Create custom exception classes for better error handling:

In [ ]:
class DatabaseError(Exception):
    """Base class for database-related errors"""
    pass

class UserAlreadyExistsError(DatabaseError):
    """Raised when trying to create a user that already exists"""
    pass

class InvalidUserDataError(DatabaseError):
    """Raised when user data is invalid"""
    pass

def create_user(username, email, age):
    """Create a user with custom error handling"""
    
    # Validate input
    if not username or not isinstance(username, str):
        raise InvalidUserDataError('Username must be a non-empty string')
    
    if not email or '@' not in email:
        raise InvalidUserDataError('Valid email is required')
    
    if not isinstance(age, int) or age < 0 or age > 150:
        raise InvalidUserDataError('Age must be an integer between 0 and 150')
    
    try:
        with conn:
            cursor.execute('INSERT INTO users (username, email, age) VALUES (?, ?, ?)', 
                           (username, email, age))
            return True
            
    except sqlite3.IntegrityError as e:
        if 'username' in str(e):
            raise UserAlreadyExistsError(f'User {username} already exists')
        elif 'email' in str(e):
            raise UserAlreadyExistsError(f'Email {email} already exists')
        else:
            raise DatabaseError(f'Integrity constraint violated: {e}')
            
    except sqlite3.Error as e:
        raise DatabaseError(f'Database error: {e}')

print('Custom error classes and create_user function defined')

# Test custom error handling
test_cases = [
    ('valid_user', 'valid@example.com', 25),
    ('alice', 'duplicate@example.com', 30),  # Duplicate username
    ('another', 'alice@example.com', 30),    # Duplicate email
    ('', 'empty@example.com', 25),           # Invalid username
    ('invalid', 'invalid_email', 25),        # Invalid email
    ('age_test', 'age@example.com', 200)     # Invalid age
]

for username, email, age in test_cases:
    try:
        create_user(username, email, age)
        print(f'✓ Created user: {username}')
    except UserAlreadyExistsError as e:
        print(f'✗ User already exists: {e}')
    except InvalidUserDataError as e:
        print(f'✗ Invalid data: {e}')
    except DatabaseError as e:
        print(f'✗ Database error: {e}')

## Logging Errors

Proper logging is important for debugging and monitoring:

In [ ]:
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def database_operation_with_logging():
    """Database operation with comprehensive logging"""
    logger.info('Starting database operation')
    
    try:
        with conn:
            cursor.execute('INSERT INTO users (username, email, age) VALUES (?, ?, ?)', 
                           ('logged_user', 'logged@example.com', 35))
            
            logger.info('Database operation completed successfully')
            return True
            
    except sqlite3.IntegrityError as e:
        logger.error(f'Integrity error: {e}')
        return False
        
    except sqlite3.OperationalError as e:
        logger.error(f'Operational error: {e}')
        return False
        
    except Exception as e:
        logger.error(f'Unexpected error: {e}', exc_info=True)
        return False

print('Logging function defined')

# Test with logging
result = database_operation_with_logging()
print(f'Logged operation result: {result}')

## Best Practices for Error Handling

- **Always use try-except blocks** around database operations
- **Catch specific exceptions** first, then general ones
- **Always rollback transactions** on errors
- **Use context managers** for automatic resource management
- **Validate input** before database operations
- **Log errors** with appropriate levels and context
- **Create custom exceptions** for application-specific errors
- **Test error scenarios** to ensure proper handling
- **Don't expose internal errors** to end users
- **Gracefully degrade** when database is unavailable

In [ ]:
# Final check of database state
cursor.execute('SELECT COUNT(*) FROM users')
total_users = cursor.fetchone()[0]

cursor.execute('SELECT username, email, age FROM users ORDER BY username')
all_users = cursor.fetchall()

print(f'\nFinal database state: {total_users} users')
print('All users:')
for user in all_users:
    print(f'  {user[0]}: {user[1]} (age {user[2]})')

# Clean up
conn.commit()

## Summary

In this notebook, you learned:

- The SQLite exception hierarchy and different error types
- How to handle constraint violations (UNIQUE, NOT NULL, CHECK)
- Proper transaction error handling with rollback
- Using context managers for automatic error handling
- Creating custom exception classes for better error management
- Logging database errors and operations
- Best practices for robust database error handling

Proper error handling is essential for creating reliable database applications. Always anticipate potential errors and handle them gracefully to maintain data integrity and provide good user experience.

In [ ]:
# Close the connection
conn.close()
print('Database connection closed')